Pinn to solve lorentz equation

first we import the data set and implement a way to sample the magnetic field

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

# Load the magnetic field data from a CSV file
data = pd.read_csv(
    "results.csv"
)  # Ensure the file has columns: x, y, z, Bx, By, Bz
positions = data[["x", "y", "z"]].values  # Position data
fields = data[["Bx", "By", "Bz"]].values  # Magnetic field components

# Create a k-d tree for efficient nearest-neighbor queries
tree = cKDTree(positions)


def magnetic_field(position):
    """
    Query the magnetic field for a given position.
    Args:
        position (list or np.array): [x, y, z] position in space.
    Returns:
        np.array: [Bx, By, Bz] magnetic field vector at the queried position.
    """
    _, idx = tree.query(position)  # Find nearest neighbor
    return fields[idx]

In [2]:
data.head(10)

,x,y,z,Bx,By,Bz
0,0.03139,0.02795,-0.03004,0.002615,3.259720e-03,0.003193
1,-0.04792,-0.01556,0.04822,0.002225,1.149863e-03,0.000000
2,-0.00440,-0.01597,0.02577,0.006843,2.228426e-02,-0.008239
3,0.04945,-0.03221,-0.01103,0.000000,-1.110008e-03,0.007755
4,-0.00804,-0.01265,-0.03829,-0.001783,-3.266066e-03,-0.007275
5,-0.02229,-0.02712,0.00005,-0.002743,-4.353262e-03,0.016515
6,0.01144,0.04998,-0.00289,-0.000730,-3.433735e-19,0.011642
7,-0.00584,-0.01543,-0.00063,-0.010433,-1.899825e-02,0.137786
8,-0.03476,-0.04805,0.04129,0.002248,0.000000e+00,0.002022
9,-0.01297,-0.04224,0.00540,0.000193,5.680308e-04,0.010637


In [6]:
import torch
import torch.nn as nn
m = 1.67262192369e-27  # proton mass in kg
q = 1.602176634e-19  # proton charge in Coulombs (or +ve if you want a proton)


class TrajectoryPINN(nn.Module):
    """
    Physics-Informed Neural Network (PINN) for predicting the trajectory of a charged particle.
    This network is designed to take a single time input and predict the spatial coordinates
    (x, y, z) of the particle at that time.
    """

    def __init__(self):
        """
        Initialize the neural network structure.
        The network consists of a series of fully connected (linear) layers with Tanh activation functions.
        """
        super().__init__()  # Call the parent class (nn.Module) constructor

        # Define the network architecture using nn.Sequential.
        # The architecture includes:
        # - An input layer that takes a single value (time t)
        # - Three hidden layers with 128 neurons each and Tanh activation functions
        # - An output layer with 3 neurons, representing the x(t), y(t), and z(t) coordinates
        self.net = nn.Sequential(
            nn.Linear(
                1, 128
            ),  # Input layer: maps 1 input (time t) to 128 hidden neurons
            nn.Tanh(),  # Activation function: Tanh introduces non-linearity
            nn.Linear(
                128, 128
            ),  # Hidden layer: maps 128 neurons to another 128 neurons
            nn.Tanh(),  # Activation function: Tanh
            nn.Linear(128, 128),  # Another hidden layer with 128 neurons
            nn.Tanh(),  # Activation function: Tanh
            nn.Linear(128, 3),  # Output layer: maps 128 neurons to 3 outputs (x, y, z)
        )

    def forward(self, t):
        """
        Perform the forward pass of the network.

        Args:
            t (Tensor): A tensor containing the time input(s) for which to predict the trajectory.
                        Shape: (batch_size, 1), where each row is a single time value.

        Returns:
            Tensor: A tensor containing the predicted (x, y, z) coordinates for the input times.
                    Shape: (batch_size, 3), where each row represents [x(t), y(t), z(t)].
        """
        return self.net(t)  # Pass the input t through the network and return the output

In [12]:
def physics_loss(model, t, s0, v0, device):
    """
    Compute the loss function for the PINN using the Lorentz force equation.
    Args:
        model (nn.Module): The PINN model.
        t (torch.Tensor): Time values, shape: (N, 1).
        s0 (torch.Tensor): Initial position [x0, y0, z0].
        v0 (torch.Tensor): Initial velocity [vx0, vy0, vz0].
        device (torch.device): Device (CPU or GPU).
    Returns:
        torch.Tensor: The total loss value.
    """
    # Enable autograd for time
    t.requires_grad = True

    # Predicted positions, shape: (N, 3)
    s = model(t)

    # Check shape of s
    if s.shape[1] != 3:
        raise ValueError(f"Expected model output to have shape (N, 3), got {s.shape}")

    # ------------------------------------------------------------------
    # Compute velocity v = ds/dt by taking partial derivatives of each
    # dimension of s w.r.t. t individually, then stacking them.
    # ------------------------------------------------------------------
    v_components = []
    for i in range(3):
        # s[:, i] has shape (N,)
        # We'll compute d(s[:, i]) / dt => shape (N,)
        grad_i = torch.autograd.grad(
            s[:, i],  # outputs
            t,  # inputs
            grad_outputs=torch.ones_like(s[:, i]),
            create_graph=True,
        )[
            0
        ]  # shape: (N, 1)

        v_components.append(grad_i.reshape(-1))  # make it (N,)

    # Stack into (N, 3)
    v = torch.stack(v_components, dim=1)
    # Debugging: Check velocity shape
    if v.shape[1] != 3:
        raise ValueError(f"Expected velocity to have shape (N, 3), got {v.shape}")

    # ------------------------------------------------------------------
    # Compute acceleration a = dv/dt similarly.
    # ------------------------------------------------------------------
    a_components = []
    for i in range(3):
        grad_i = torch.autograd.grad(
            v[:, i], t, grad_outputs=torch.ones_like(v[:, i]), create_graph=True
        )[0]

        a_components.append(grad_i.reshape(-1))

    a = torch.stack(a_components, dim=1)
    # Debugging: Check acceleration shape
    if a.shape[1] != 3:
        raise ValueError(f"Expected acceleration to have shape (N, 3), got {a.shape}")

    # Query magnetic field at predicted positions
    B_values = torch.stack(
        [
            torch.tensor(
                magnetic_field(pos.cpu().detach().numpy()), dtype=torch.float32
            )
            for pos in s
        ]
    ).to(
        device
    )  # Shape: (N, 3)
    #print(B_values)
    # Debugging: Check B_values shape
    if B_values.shape[1] != 3:
        raise ValueError(
            f"Expected B_values to have shape (N, 3), got {B_values.shape}"
        )

    # Lorentz force residual
    lorentz_residual = m * a - q * torch.cross(v, B_values, dim=1)
    physics_loss_val = torch.mean(lorentz_residual**2)

    # Initial conditions loss
    ic_loss = torch.mean((s[0] - s0) ** 2) + torch.mean((v[0] - v0) ** 2)
   # print("current physics loss")
    #print(physics_loss_val)
    #print("current ic loss")
    #print(ic_loss)
    return physics_loss_val + 1.2* ic_loss

In [13]:
# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TrajectoryPINN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=12e-4)

# Initial conditions
s0 = torch.tensor([[0.0, 0.007, -0.05]], device=device)  # Initial position
v0 = torch.tensor([[0.0, 0.0, 1e6]], device=device)  # Initial velocity (z-direction)
t_train = torch.linspace(0, 1e-5, 1000, device=device).unsqueeze(1)  # Time values

# Training loop
for epoch in range(50000):
    optimizer.zero_grad()
    loss = physics_loss(model, t_train, s0, v0, device)
    loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Save the trained model
torch.save(model.state_dict(), "trajectory_pinn_3d.pth")
print("Training complete and model saved.")

current physics loss
tensor(6.1657e-44, grad_fn=<MeanBackward0>)
current ic loss
tensor(3.3333e+11, grad_fn=<AddBackward0>)
Epoch 0, Loss: 333333331968.0
current physics loss
tensor(1.4013e-44, grad_fn=<MeanBackward0>)
current ic loss
tensor(3.3333e+11, grad_fn=<AddBackward0>)
current physics loss
tensor(1.5134e-43, grad_fn=<MeanBackward0>)
current ic loss
tensor(3.3333e+11, grad_fn=<AddBackward0>)
current physics loss
tensor(3.4052e-43, grad_fn=<MeanBackward0>)
current ic loss
tensor(3.3333e+11, grad_fn=<AddBackward0>)
current physics loss
tensor(6.9785e-43, grad_fn=<MeanBackward0>)
current ic loss
tensor(3.3333e+11, grad_fn=<AddBackward0>)
current physics loss
tensor(1.2892e-42, grad_fn=<MeanBackward0>)
current ic loss
tensor(3.3333e+11, grad_fn=<AddBackward0>)
current physics loss
tensor(1.2107e-42, grad_fn=<MeanBackward0>)
current ic loss
tensor(3.3333e+11, grad_fn=<AddBackward0>)
current physics loss
tensor(2.0067e-42, grad_fn=<MeanBackward0>)
current ic loss
tensor(3.3333e+11, gr

KeyboardInterrupt: 